In [4]:
from jsonql import smart_open, read_jsons
import os
import matplotlib.pyplot as plt
import perplexity
from pathlib import Path
from sentence_splitter import SentenceSplitter, split_text_into_sentences
import re
import json
from tqdm import tqdm
from multiprocessing import Pool
import itertools
import numpy as np
import subprocess
import pandas as pd
from tqdm import tqdm
import bokeh.core.enums
import operator

In [5]:
file = "/checkpoint/guw/cc_clean2/lid_exp/0000.json.gz"

In [6]:
count_total = {}
matrix_confusion = {}
langs1 = []
langs2 = []

In [7]:
# source = smart_open(file)
# docs = read_jsons(source)
# for i,doc in enumerate(docs):
#     lid_after_dedup = doc['lid_after_dedup']
#     lang1, score1 = next(iter(lid_after_dedup.items()))
#     if score1 < 0.5:
#         lang1 = 'none'
#     lid_before_dedup = doc['lid_before_dedup']
#     lang2, score2 = next(iter(lid_before_dedup.items()))
#     if score2 < 0.5:
#         lang2 = 'none'
    
#     if lang1 not in langs1:
#         langs1.append(lang1)
#     if lang2 not in langs2:
#         langs2.append(lang2)
    
#     if lang1 in count_total:
#         count_total[lang1] += 1
#         if lang2 in matrix_confusion[lang1]:
#             matrix_confusion[lang1][lang2] +=1
#         else:
#             matrix_confusion[lang1][lang2] =1
#     else :
#         count_total[lang1] = 1
#         matrix_confusion[lang1] = {lang2:1}
    
    

In [8]:
# tmp = json.dumps(count_total)
# f = open("/private/home/malachaux/ccall/count_total.json","w")
# f.write(tmp)
# f.close()

# tmp = json.dumps(matrix_confusion)
# f = open("/private/home/malachaux/ccall/matrix_confusion.json","w")
# f.write(tmp)
# f.close()

In [9]:
with open('/private/home/malachaux/ccall/count_total.json', 'r') as fp:
    count_total = json.load(fp)

with open('/private/home/malachaux/ccall/matrix_confusion.json', 'r') as fp:
    matrix_confusion = json.load(fp)

In [10]:
languages_sorted = sorted(count_total.items(), key=operator.itemgetter(1), reverse = True)
languages_sorted_reverse = sorted(count_total.items(), key=operator.itemgetter(1))

languages_sorted = [l for l, _ in languages_sorted ]
languages_sorted_reverse = [l for l, _ in languages_sorted_reverse ]

In [11]:
normalized_matrix_confusion = []
for lang in languages_sorted :
    tab = []
    for lang2 in languages_sorted  :
        if lang2 in  matrix_confusion[lang].keys():
            tab.append(matrix_confusion[lang][lang2]/count_total[lang])
        else:
            tab.append(0)
    normalized_matrix_confusion.append(tab)

In [12]:
len(normalized_matrix_confusion)

121

In [13]:
matrix = [normalized_matrix_confusion[-i] for i in range(1,122)]

In [14]:
normalized_matrix_confusion = np.array(normalized_matrix_confusion)
matrix = np.array(matrix)                              

In [20]:
import pandas as pd
import math
from bokeh.io import output_file, show, export_png
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, ColumnDataSource, PrintfTickFormatter
from bokeh.plotting import figure, output_notebook
from bokeh.transform import transform
from bokeh.palettes import Viridis256


df = pd.DataFrame(
    matrix[:50],
    columns=languages_sorted,
    index=languages_sorted_reverse[:50])
df.index.name = 'LID_before_dedup'
df.columns.name = 'LID_after_dedup'
#To use my solution you have to stack the data.frame :

# Prepare data.frame in the right format
df = df.stack().rename("value").reset_index()

# You can use your own palette here
colors = Viridis256

# Had a specific mapper to map color with value
mapper = LinearColorMapper(
    palette=colors, low=df.value.min(), high=df.value.max())
# Define a figure
p = figure(
    plot_width=1200,
    plot_height=500,
    title="LID before and after dedup",
    x_range=languages_sorted,
    y_range=languages_sorted_reverse[:50],
    x_axis_location="above",
    tools="save")

p.xaxis.major_label_orientation = math.pi/2
p.xaxis.axis_label = "lang id before dedup"
p.yaxis.axis_label = "lang id after dedup"
p.yaxis.axis_label_text_font_style = 'normal'
p.xaxis.axis_label_text_font_style = 'normal'

# Create rectangle for heatmap
p.rect(
    x="LID_after_dedup",
    y="LID_before_dedup",
    width=1,
    height=1,
    source=ColumnDataSource(df),
    line_color=None,
    fill_color=transform('value', mapper))
# Add legend
color_bar = ColorBar(
    color_mapper=mapper,
    location=(0, 0),
    ticker=BasicTicker(desired_num_ticks=10))

p.add_layout(color_bar, 'right')
output_notebook() 
show(p)

Loading BokehJS ...